In [1]:
from transformers import WhisperProcessor, WhisperForConditionalGeneration
import torch.nn.functional as F
import torch
import torchaudio


In [2]:

# 모델 및 프로세서 로드
processor = WhisperProcessor.from_pretrained("openai/whisper-small")
model = WhisperForConditionalGeneration.from_pretrained("openai/whisper-small")



In [3]:
from datasets import load_dataset, Audio


dataset = load_dataset("CAiRE/ASCEND")

In [4]:
# 음성 파일 로드 및 전처리
audio_data = dataset['train'][1]['audio']  # audio 데이터 가져오기
waveform = audio_data["array"]  # waveforms 추출
sample_rate = audio_data["sampling_rate"]  # 샘플링 레이트 추출

# 다중 채널 오디오를 단일 채널로 변환 (평균)
if len(waveform.shape) > 1 and waveform.shape[0] > 1:
    waveform = waveform.mean(axis=0)

# 샘플링 레이트 변환
waveform = torchaudio.transforms.Resample(orig_freq=sample_rate, new_freq=16000)(torch.tensor(waveform))
waveform_np = waveform.numpy()

# 입력 데이터 생성
inputs = processor(waveform_np, sampling_rate=16000, return_tensors="pt")

# 언어 프롬프트 설정
forced_decoder_ids_en = processor.get_decoder_prompt_ids(language="english", task="transcribe")
forced_decoder_ids_zh = processor.get_decoder_prompt_ids(language="chinese", task="transcribe")


In [5]:
# 영어 logits 계산
with torch.no_grad():
    outputs_en = model.generate(
        inputs.input_features,
        forced_decoder_ids=forced_decoder_ids_en,
        return_dict_in_generate=True,
        output_scores=True
    )
logits_en = outputs_en.scores  # 영어 확률 분포

# 한국어 logits 계산
with torch.no_grad():
    outputs_zh = model.generate(
        inputs.input_features,
        forced_decoder_ids=forced_decoder_ids_zh,
        return_dict_in_generate=True,
        output_scores=True
    )
logits_zh = outputs_zh.scores  # 한국어 확률 분포



/Users/jpark/.pyenv/versions/3.11.10/lib/python3.11/site-packages/transformers/generation/configuration_utils.py:774: UserWarning: `return_dict_in_generate` is NOT set to `True`, but `output_scores` is. When `return_dict_in_generate` is not `True`, `output_scores` is ignored.
  warnings.warn(
The attention mask is not set and cannot be inferred from input because pad token is same as eos token. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
From v4.47 onwards, when a model cache is to be returned, `generate` will return a `Cache` instance instead by default (as opposed to the legacy tuple of tuples format). If you want to keep returning the legacy format, please set `return_legacy_cache=True`.


In [7]:
# Code-switching 비교
code_switching_result = []
for frame_idx in range(len(logits_en)):
    english_prob = logits_en[frame_idx].max().item()
    chinese_prob = logits_zh[frame_idx].max().item()

    if english_prob > chinese_prob:
        selected_lang = "English"
        selected_word = processor.tokenizer.decode(logits_en[frame_idx].argmax().item())
    else:
        selected_lang = "Chinese"
        selected_word = processor.tokenizer.decode(logits_zh[frame_idx].argmax().item())

    code_switching_result.append((frame_idx, selected_lang, selected_word))

# 결과 출력
print(f"original transcription: {dataset['train'][1]['transcription']}")
for frame, lang, word in code_switching_result:
    print(f"Frame {frame}: {lang} -> {word}")

original transcription: 嗯hello我的名字叫徐妍
Frame 0: English ->  Hello
Frame 1: English -> ,
Frame 2: English ->  my
Frame 3: Chinese -> 名
Frame 4: English ->  is
Frame 5: Chinese -> 叫
Frame 6: English ->  Yan
Frame 7: English -> .
Frame 8: English -> <|endoftext|>
